# **네이버 영화리뷰 감정분석 with Hugging Face BERT**

BERT(Bidirectional Encoder Representations from Transformers)는 구글이 개발한 사전훈련(pre-training) 모델입니다. 위키피디아 같은 텍스트 코퍼스를 사용해서 미리 학습을 하면, 언어의 기본적인 패턴을 이해한 모델이 만들어집니다. 이를 기반으로 새로운 문제에 적용하는 전이학습(transfer learning)을 수행합니다. 좀 더 적은 데이터로 보다 빠르게 학습이 가능하다는 장점이 있습니다. 그래서 최근 자연어처리의 핵심 기법으로 떠오르고 있습니다.

이 예제에서는 한글 NLP의 Hello world라고 할 수 있는 네이버 영화리뷰 감정분석을 구현해보겠습니다. 가장 유명한 모델 중 하나인 Hugging Face의 PyTorch BERT를 사용하였습니다. 아래의 Chris McCormick의 블로그를 참조하여 한글에 맞게 수정하였음을 미리 알려드립니다.

< BERT Fine-Tuning Tutorial with PyTorch ><br>
-> https://mccormickml.com/2019/07/22/BERT-fine-tuning
<br>
<br>
<br>
BERT에 대해서 좀 더 자세한 설명은 박상길님과 Jay Alammar의 블로그를 참조하시기 바랍니다.

< BERT 톺아보기 ><br>
-> http://docs.likejazz.com/bert/

< The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning) ><br>
-> http://jalammar.github.io/illustrated-bert/
<br>
<br>


In [1]:
!pip install tensorflow
!pip install torch
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13355691065404306724
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14738522112
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12593268122810232203
 physical_device_desc: "device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:03:00.0, compute capability: 7.0"
 xla_global_id: 416903419]

<br>
<br>

# **준비 사항**

In [2]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers
# !pip install pytorch-transformers

In [1]:

import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

<br>
<br>

# **데이터 로드**

In [4]:
# 디렉토리의 파일 목록
!ls nsmc -la

total 38636
drwxrwxr-x  5 team2 team2     4096 Mar 15 16:16 .
drwxr-xr-x 16 team2 team2     4096 Mar 17 13:09 ..
drwxrwxr-x  2 team2 team2     4096 Mar 15 16:16 code
drwxrwxr-x  8 team2 team2     4096 Mar 15 16:16 .git
-rw-rw-r--  1 team2 team2  4893335 Mar 15 16:16 ratings_test.txt
-rw-rw-r--  1 team2 team2 14628807 Mar 15 16:16 ratings_train.txt
-rw-rw-r--  1 team2 team2 19515078 Mar 15 16:16 ratings.txt
drwxrwxr-x  2 team2 team2   462848 Mar 15 16:16 raw
-rw-rw-r--  1 team2 team2     2596 Mar 15 16:16 README.md
-rw-rw-r--  1 team2 team2    36746 Mar 15 16:16 synopses.json


In [4]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
# train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
# test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

train = pd.read_csv("./data/train.csv", encoding = 'UTF-8')
test = pd.read_csv("./data/DongA.csv", encoding = 'UTF-8', sep='\n')


print(train.shape)
print(test.shape)

(119478, 3)
(449, 1)


In [5]:
print(train['label'].unique())
print(test['label'].unique())

[0 1 2]


KeyError: 'label'

훈련셋 150,000개와 테스트셋 50,000개의 데이터가 존재합니다.
<br>
<br>
<br>

In [7]:
# 훈련셋의 앞부분 출력
# train = train.drop(['id'], axis = 1)
train.head(10)

,Unnamed: 0,title,label
0,9,한국당 손혜원 추진 박물관 나전칠기 판매처 의혹,0
1,25,정치인 없이 핵심부대 꾸린 황교안‧오세훈 부족한 2%는,0
2,43,갈수록 심해지는 의료진 대상 범죄 대책 마련 시급,0
3,48,아베 총리 화기관제 레이더 조준 위험한 행위 재발방지책 마련돼야,0
4,80,자동차 타이어 석유화학 부진 중소기업 어려움 가중,0
5,88,미국판 복면가왕 2일 FOX서 첫방 의상 한 벌 2억 원,0
6,94,교복값 비싼 이유 있었네 브랜드 업체끼리 담합,0
7,115,과거사규명법 또 불투명 화가 난다 피해자들 울분,0
8,116,옷 로비 의혹 무죄 판례로 본 김태우 靑비밀누설,0
9,121,단독 檢 징용소송 거래 의혹 수사 고삐 김규현 전 靑수석 등 소환,0


In [8]:
# 훈련셋의 앞부분 출력
# train = pd.concat([train,train2 ])

print(train.shape)
train.tail(10)

(119478, 3)


,Unnamed: 0,title,label
119468,48328,축하금 지원해 저출산 극복 돕는 지역 교회,2
119469,48329,지식재산 창출 4개 사업에 30억원 지원,2
119470,48330,한국축구 中 완파하고 우승 향한 꽃길 걷는다,2
119471,48331,영월 2 353명 노인일자리사업 61억 투입,2
119472,48332,속초 러 연해주 백두산 항로 5년만에 열린다,2
119473,48333,정선 축구장 2천개 면적 조상 땅 찾아줬다,2
119474,48334,드론 날리고 열화상카메라 찍고 미세먼지 잡아라,2
119475,48335,시 군의장협의회 미국 연수 결국 취소,2
119476,48336,홍천 홍천군 교육청 미래인재 육성 머리맞대,2
119477,48337,기업총수들에 규제혁신 약속,2


In [6]:
# test = pd.concat([test,test2])
print(test.shape)

test.tail(10)

# print(test['label'].unique())

(449, 1)


,[스타트업 in 과기대] 테라블록 “저급 폐플라스틱도 무한 재생 가능한 자원 된다”
439,국내 최대 마라톤에 참가자 몰렸다… ‘2022 서울마라톤’ 5일만에 모집 마감
440,“날 풀리자 바빠진 반려인들”… ‘댕댕이·냥이’ 용품 판매량 급증
441,尹 “독소조항 폐지” 공약에…공수처 “지속 시행돼야” 반대
442,"바이든, 시진핑에 “러 지원 말라” 요청할 듯…오늘 밤 통화"
443,윤석열 “정부 초기 모습 보면 임기 말도 알 수 있어”
444,우리은행 전세대출 한도 늘렸다…KB-신한도 완화 검토
445,"尹 “국민 눈높이”…인수위 53일, 정부 초기 ‘성패 좌우’"
446,여성만 골라 비비탄 쏜 30대 남성…“남자보다 반응 커서”
447,“당연히 도울 일”…몰카범 ‘멱살’ 잡혀도 끝까지 제압한 쿠팡 기사
448,임종석 “의견수렴-예산편성 없이 靑 이전하나”


In [10]:
# test2 = test2.rename(columns = {'Unnamed: 0' : 'id'})

# test = pd.concat([test, test2])
# test.tail(10)

id는 회원정보, document는 리뷰 문장입니다. label이 0이면 부정, 1이면 긍정으로 분류됩니다. id는 사용하지 않기 때문에 document와 label만 추출하겠습니다. 

<br>
<br>

# **전처리 - 훈련셋**

In [11]:
# 리뷰 문장 추출
train = train.rename(columns = {'title' : 'document'})
sentences = train['document']
sentences[:10]

0              한국당 손혜원 추진 박물관 나전칠기 판매처 의혹
1          정치인 없이 핵심부대 꾸린 황교안‧오세훈 부족한 2%는
2             갈수록 심해지는 의료진 대상 범죄 대책 마련 시급
3     아베 총리 화기관제 레이더 조준 위험한 행위 재발방지책 마련돼야
4             자동차 타이어 석유화학 부진 중소기업 어려움 가중
5         미국판 복면가왕 2일 FOX서 첫방 의상 한 벌 2억 원
6               교복값 비싼 이유 있었네 브랜드 업체끼리 담합
7              과거사규명법 또 불투명 화가 난다 피해자들 울분
8              옷 로비 의혹 무죄 판례로 본 김태우 靑비밀누설
9    단독 檢 징용소송 거래 의혹 수사 고삐 김규현 전 靑수석 등 소환
Name: document, dtype: object

In [12]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 한국당 손혜원 추진 박물관 나전칠기 판매처 의혹 [SEP]',
 '[CLS] 정치인 없이 핵심부대 꾸린 황교안‧오세훈 부족한 2%는 [SEP]',
 '[CLS] 갈수록 심해지는 의료진 대상 범죄 대책 마련 시급 [SEP]',
 '[CLS] 아베 총리 화기관제 레이더 조준 위험한 행위 재발방지책 마련돼야 [SEP]',
 '[CLS] 자동차 타이어 석유화학 부진 중소기업 어려움 가중 [SEP]',
 '[CLS] 미국판 복면가왕 2일 FOX서 첫방 의상 한 벌 2억 원 [SEP]',
 '[CLS] 교복값 비싼 이유 있었네 브랜드 업체끼리 담합 [SEP]',
 '[CLS] 과거사규명법 또 불투명 화가 난다 피해자들 울분 [SEP]',
 '[CLS] 옷 로비 의혹 무죄 판례로 본 김태우 靑비밀누설 [SEP]',
 '[CLS] 단독 檢 징용소송 거래 의혹 수사 고삐 김규현 전 靑수석 등 소환 [SEP]']

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력은 위의 그림과 같은 형식입니다. Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입됩니다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 합니다. [SEP]은 Seperation을 가리키는데, 두 문장을 구분하는 역할을 합니다. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 넣습니다.
<br>
<br>
<br>

In [13]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 0, 0, ..., 2, 2, 2])

In [14]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 한국당 손혜원 추진 박물관 나전칠기 판매처 의혹 [SEP]
['[CLS]', '한국', '##당', '손', '##혜', '##원', '추', '##진', '박', '##물', '##관', '나', '##전', '##칠', '##기', '판', '##매', '##처', '의', '##혹', '[SEP]']


BERT는 형태소분석으로 토큰을 분리하지 않습니다. WordPiece라는 통계적인 방식을 사용합니다. 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만듭니다. 이렇게 하면 언어에 상관없이 토큰을 생성할 수 있다는 장점이 있습니다. 또한 신조어 같이 사전에 없는 단어를 처리하기도 좋습니다. 

위의 결과에서 ## 기호는 앞 토큰과 이어진다는 표시입니다. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용합니다. 그래서 한글도 처리가 가능합니다.
<br>
<br>
<br>

In [15]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  48556,  21928,   9450, 119437,  14279,   9765,  18623,
         9319,  29364,  20595,   8982,  16617, 119284,  12310,   9903,
       100372,  60469,   9637, 119438,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

보통 딥러닝 모델에는 토큰 자체를 입력으로 넣을 수 없습니다. 임베딩 레이어에는 토큰을 숫자로 된 인덱스로 변환하여 사용합니다. BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어사전을 가지고 있습니다. 이를 참조하여 토큰을 인덱스로 바꿔줍니다.
<br>
<br>
<br>

In [16]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [17]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9764, 119168,   8857,  78136,  27355, 118922,   9521,  30858,
         22458,  66324,   9100, 119014,  11102,   9405,  25486,  30005,  25503,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [18]:
# 배치 사이즈
batch_size = 128

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<br>
<br>

# **전처리 - 테스트셋**

In [19]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                      소의 해 정치인들 싸움 좀 그만하시죠
1                  지금 네 울음이 나중에 웃음이 될 수도 있어
2                        고독한 양색시 김수임의 크리스마스
3    화면의 사실이 현장의 진실과 달라 신경민 앵커 KBS 보신각방송 비판
4                    맞아죽지 않으려면 한국교회 잘못 비판하라
5                   이율배반적인 악성 관계 는 우리들의 자화상
6                     국회 내 경찰 동원 의혹 파문 커질 듯
7                          시체 위를 달렸다 아 미안하다
8                         우울증 극복에 관한 희망 보고서
9                  다른 팀 가는 선수 살해할 것 과격한 축구팬
Name: document, dtype: object

In [20]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 소의 해 정치인들 싸움 좀 그만하시죠 [SEP]',
 '[CLS] 지금 네 울음이 나중에 웃음이 될 수도 있어 [SEP]',
 '[CLS] 고독한 양색시 김수임의 크리스마스 [SEP]',
 '[CLS] 화면의 사실이 현장의 진실과 달라 신경민 앵커 KBS 보신각방송 비판 [SEP]',
 '[CLS] 맞아죽지 않으려면 한국교회 잘못 비판하라 [SEP]',
 '[CLS] 이율배반적인 악성 관계 는 우리들의 자화상 [SEP]',
 '[CLS] 국회 내 경찰 동원 의혹 파문 커질 듯 [SEP]',
 '[CLS] 시체 위를 달렸다 아 미안하다 [SEP]',
 '[CLS] 우울증 극복에 관한 희망 보고서 [SEP]',
 '[CLS] 다른 팀 가는 선수 살해할 것 과격한 축구팬 [SEP]']

In [21]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 0, ..., 2, 2, 2])

In [22]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 소의 해 정치인들 싸움 좀 그만하시죠 [SEP]
['[CLS]', '소', '##의', '해', '정', '##치', '##인', '##들', '싸', '##움', '좀', '그', '##만', '##하', '##시', '##죠', '[SEP]']


In [23]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9448,  10459,   9960,   9670,  18622,  12030,  27023,
         9496, 119169,   9682,   8924,  19105,  35506,  14040, 119217,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [24]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [25]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9448,  10459,   9960,   9670,  18622,  12030,  27023,   9496,
        119169,   9682,   8924,  19105,  35506,  14040, 119217,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [26]:
# 배치 사이즈
batch_size = 128

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [27]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [28]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-PCIE-32GB


In [29]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

![대체 텍스트](http://www.mccormickml.com/assets/BERT/padding_and_mask.png)

사전훈련된 BERT는 다양한 문제로 전이학습이 가능합니다. 여기서는 위의 그림과 같이 한 문장을 분류하는 방법을 사용합니다. 영화리뷰 문장이 입력으로 들어가면, 긍정/부정으로 구분합니다. 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 파인튜닝을 합니다. Huggning Face는 BertForSequenceClassification() 함수를 제공하기 때문에 쉽게 구현할 수 있습니다.
<br>
<br>
<br>

In [30]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/home/team2/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<br>
<br>

# **모델 학습**

In [31]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

에폭마다 훈련셋과 검증셋을 반복하여 학습을 수행합니다. 

<br>
<br>

# **테스트셋 평가**

In [53]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    353.    Elapsed: 0:00:27.
  Batch   200  of    353.    Elapsed: 0:00:54.
  Batch   300  of    353.    Elapsed: 0:01:22.

Accuracy: 0.94
Test took: 0:01:36


테스트셋의 정확도가 87%입니다. <BERT 톺아보기> 블로그에서는 같은 데이터로 88.7%를 달성하였습니다. 거기서는 한글 코퍼스로 사전훈련을 하여 새로운 모델을 만들었습니다. 반면에 우리는 BERT의 기본 모델인 bert-base-multilingual-cased를 사용했기 때문에 더 성능이 낮은 것 같습니다.

<br>
<br>

# **새로운 문장 테스트**

In [42]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [43]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [44]:
def softmax(a) :
  e_a = np.exp(a - np.max(a))
  return e_a / e_a.sum()

In [52]:
logits = test_sentences(['새해에는 건강 좀 챙기자'])
print(np.argmax(logits))
print(softmax(logits))

2
[[0.24301308 0.02063552 0.7363514 ]]


In [46]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))
print(softmax(logits))

[[ 5.1133394 -3.3015823 -2.3953846]]
0
[[9.9923086e-01 2.2136644e-04 5.4785842e-04]]


In [47]:
from scipy.special import softmax

In [ ]:
# 모델 저장
# https://tutorials.pytorch.kr/beginner/saving_loading_models.html

model_PATH = "/home/team2/model/"

torch.save(model.state_dict(), model_PATH + "model_state_dict.pt")
torch.save(optimizer.state_dict(), model_PATH + "optimizer_state_dict.pt")

In [55]:
np.set_printoptions(precision=6, suppress=True)
 
pd.options.display.float_format = '{:.5f}'.format
pd.reset_option('display.float_format')

In [56]:
from tqdm import tqdm
import csv

# ttest = pd.read_csv("./data/test.csv", encoding = 'UTF-8')

f = open('./data/test.csv', 'r')
rdr = csv.reader(f)

result = []
document = []
label = []

# ttest

for i in tqdm(rdr):
#     print(i)
    logits = test_sentences([i[1]])
    document.append(i[1])
    label.append(i[2])
    result.append(softmax(logits))

dff = pd.DataFrame({"document": document, "result": result , "label" : label})

45091it [06:44, 111.38it/s]


In [57]:
dff

,document,result,label
0,document,"[[0.13219683, 0.2396009, 0.6282024]]",label
1,소의 해 정치인들 싸움 좀 그만하시죠,"[[0.99964535, 0.00015889427, 0.00019595971]]",0
2,지금 네 울음이 나중에 웃음이 될 수도 있어,"[[0.9995657, 0.00021888714, 0.00021519337]]",0
3,고독한 양색시 김수임의 크리스마스,"[[0.2968331, 0.07994594, 0.62322104]]",0
4,화면의 사실이 현장의 진실과 달라 신경민 앵커 KBS 보신각방송 비판,"[[0.9994546, 0.00017204654, 0.0003731974]]",0
...,...,...,...
45086,정몽준 세종시 정부가 대안 내면 당론 정하겠다,"[[0.00037861516, 0.0029781896, 0.99664295]]",2
45087,향기는 간 데 없고 겨울바람만 가득하네,"[[0.001337249, 0.001512357, 0.9971501]]",2
45088,세계챔피언 유명우 팽 당했나 권투계 논란,"[[0.001187138, 0.0021011373, 0.99671197]]",2
45089,4대강 합천보 오탁방지막 부분적으로 터져,"[[0.028799599, 0.003837724, 0.96736264]]",2


In [2]:
dff.to_csv('check.csv', encoding='utf-8')

NameError: name 'dff' is not defined

In [41]:
# load model, optimizer 설정

model_PATH = "/home/team2/model/"
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                       lr = 2e-5, # 학습률,
                       eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                       )

model.load_state_dict(torch.load(model_PATH + "model_state_dict.pt"))
optimizer.load_state_dict(torch.load(model_PATH + "optimizer_state_dict.pt"))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

학습한 모델을 가지고 실제 문장을 넣어봤습니다. 출력 로짓은 소프트맥스가 적용되지 않은 상태입니다. argmax로 더 높은 값의 위치를 라벨로 설정하면 됩니다. 0은 부정, 1은 긍정입니다. 위와 같이 새로운 문장에도 잘 분류를 하고 있습니다.
<br>
<br>
<br>

< 챗봇 개발자 모임 ><br>
- 페이스북 그룹에 가입하시면 챗봇에 대한 최신 정보를 쉽게 받으실 수 있습니다.
- https://www.facebook.com/groups/ChatbotDevKR/